# Master Discogs Database

In [44]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collection import collections
from artist import artist
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from time import sleep


################################################################################
## Music Stuff
################################################################################

### Master DB code
from masterdb import discConv, isKnownAlbum, directoryName
from masterdb import getSlimArtistDB, getArtistNameToIDMap
from masterdb import getMusicData, getRowByIndex

### General
from discogsBase import discogs

### Discogs
from artists import artists
from artist import artist
from discogsUtils import discogsUtils
from albums import albums
from album import album, albumURLInfo

### AllMusic
from artistsAM import artistsAM
from artistAM import artistAM
from discogsUtils import allmusicUtils

### MusicBrainz
from artistsMB import artistsMB
from artistMB import artistMB
from discogsUtils import musicbrainzUtils


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-03-09 18:21:46.590272


In [45]:
%load_ext autoreload
%autoreload

discAM   = discogs('allmusic')
artsAM   = artistsAM(discAM)
dutilsAM = allmusicUtils()

discDC   = discogs('discogs')
artsDC   = artists(discDC)
dutilsDC = discogsUtils()

discMB   = discogs('musicbrainz')
artsMB   = artistsMB(discMB)
dutilsMB = musicbrainzUtils()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Create Discogs <-> AllMusic Map

In [39]:
discSlimArtistDCDB = getSlimArtistDB(discDC, force=False)
artistDCNameToID   = getArtistNameToIDMap(discSlimArtistDCDB)
artistDCKeys       = artistDCNameToID.keys()

Current Time is Sat Feb 22, 2020 17:47:05 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Current Time is Sat Feb 22, 2020 17:47:10 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 4 seconds.
Current Time is Sat Feb 22, 2020 17:47:11 for 
=================================== Creating Artist Name To ID Map ===================================
Found 712629 ID -> Name entries
Found 654970 Name -> ID entries


In [40]:
discSlimArtistAMDB = getSlimArtistDB(discAM, force=False)
artistAMNameToID   = getArtistNameToIDMap(discSlimArtistAMDB)
artistAMKeys       = artistAMNameToID.keys()

Current Time is Sat Feb 22, 2020 17:47:23 for 
=================================== Creating Artist DB ===================================
Using previously created Slim Artist DB
Current Time is Sat Feb 22, 2020 17:47:23 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 183.0 millseconds.
Current Time is Sat Feb 22, 2020 17:47:23 for 
=================================== Creating Artist Name To ID Map ===================================
Found 36563 ID -> Name entries
Found 34497 Name -> ID entries


In [5]:
discSlimArtistMBDB = getSlimArtistDB(discMB, force=True)
artistMBNameToID   = getArtistNameToIDMap(discSlimArtistMBDB)
artistMBKeys       = artistMBNameToID.keys()

Current Time is Sun Mar 01, 2020 12:08:41 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data


ValueError: Could not find /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToName.p

In [41]:
artistAMToAlbumsData = discAM.getArtistIDToAlbumNamesData(debug=True)
print(len(artistAMToAlbumsData))

Data stored in /Volumes/Biggy/Discog/db-allmusic/ArtistIDToAlbumNames.p
Returning data from /Volumes/Biggy/Discog/db-allmusic/ArtistIDToAlbumNames.p
Loading data from /Volumes/Biggy/Discog/db-allmusic/ArtistIDToAlbumNames.p
  --> This file is 11.8MB.
Loading /Volumes/Biggy/Discog/db-allmusic/ArtistIDToAlbumNames.p
36563


In [46]:
artistDCToAlbumsData = discDC.getArtistIDToAlbumNamesData(debug=True)
print(len(artistDCToAlbumsData))

Data stored in /Volumes/Biggy/Discog/db-discogs/ArtistIDToAlbumNames.p
Returning data from /Volumes/Biggy/Discog/db-discogs/ArtistIDToAlbumNames.p
Loading data from /Volumes/Biggy/Discog/db-discogs/ArtistIDToAlbumNames.p
  --> This file is 190.8MB.
Loading /Volumes/Biggy/Discog/db-discogs/ArtistIDToAlbumNames.p
712629


# Create Dictionary Lookup Files

## Artist ID --> Ref and Name

#### AllMusic

In [46]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discAM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistAMDB = getSlimArtistDB(discAM, force=True)
discSlimArtistAMDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Mar 09, 2020 18:21:58 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/0-Metadata.p  	0 382
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/1-Metadata.p  	1 746
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/10-Metadata.p  	2 1163
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/11-Metadata.p  	3 1537
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/12-Metadata.p  	4 1940
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/13-Metadata.p  	5 2322
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/14-Metadata.p  	6 2681
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/15-Metadata.p  	7 3039
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/16-Metadata.p  	8 3431
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/17-Metadata.p  	9 3820
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/18-Metadata.p  	10 4184
/Volumes/Biggy/Discog/artists-allmu

  --> This file is 835.3kB.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToVariations.p
  --> This file is 835.3kB.
Current Time is Mon Mar 09, 2020 18:22:06 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 7 seconds.
Current Time is Mon Mar 09, 2020 18:22:06 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 38676 Artists
	Shape --> (38676, 1)
  Finding Real Artist Name
	Shape --> (38676, 3)
  Removing None Artist
	Shape --> (38676, 3)
  Finding Disc Artist Name
	Shape --> (38676, 4)
DataFrame Shape is (38676, 4)
Current Time is Mon Mar 09, 2020 18:22:07 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 1 seconds.
Saving Master Artist DB File
  --> This file is 2.9MB.


,Name,Artist,Num,DiscArtist
0000003100,Tania Maria,Tania Maria,None,Tania Maria
0000003500,Total Kaos,Total Kaos,None,Total Kaos
0000006000,Path of Debris,Path of Debris,None,Path of Debris
0000007300,Spider Monkey,Spider Monkey,None,Spider Monkey
0000015100,Sirone,Sirone,None,Sirone


#### MusicBrainz

In [50]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discMB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistMBDB = getSlimArtistDB(discMB, force=True)
discSlimArtistMBDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Mar 09, 2020 19:25:31 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/0-Metadata.p  	0 189
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/1-Metadata.p  	1 403
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/10-Metadata.p  	2 630
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/11-Metadata.p  	3 849
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/12-Metadata.p  	4 1047
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/13-Metadata.p  	5 1253
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/14-Metadata.p  	6 1472
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/15-Metadata.p  	7 1668
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/16-Metadata.p  	8 1886
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/17-Metadata.p  	9 2077
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/18-Metadata.p  	10 2279
/Vol

  --> This file is 909.6kB.
Saved data to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToName.p
  --> This file is 909.6kB.
Saving 20764 entries to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToVariations.p

Saving data to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToVariations.p
  --> This file is 979.6kB.
Saved data to /Volumes/Biggy/Discog/db-musicbrainz/ArtistIDToVariations.p
  --> This file is 979.6kB.
Current Time is Mon Mar 09, 2020 19:25:37 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 5 seconds.
Current Time is Mon Mar 09, 2020 19:25:37 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 20764 Artists
	Shape --> (20764, 1)
  Finding Real Artist Name
	Shape --> (20764, 3)
  Removing None Artist
	Shape --> (20764, 3)
  Finding Disc Artist Name
	Shape --> (20764, 4)
  Cleaning Disc Artist Name (MusicBrainz Only)
	Shape --> (20764, 4)
DataFrame Shape 

,Name,Artist,Num,DiscArtist
148150553282083715092252970758494088400,Kölner Saxophon Mafia,Kölner Saxophon Mafia,None,Kölner Saxophon Mafia
153306575689393285671340715814524277900,Afrodite (Czech group),Afrodite (Czech group),None,Afrodite
164497588684518360574139427720206551000,Pure Prairie League,Pure Prairie League,None,Pure Prairie League
192627206585753480770469159350953417800,The Righteous Brothers,The Righteous Brothers,None,The Righteous Brothers
196485629247468963190624302831920892400,Planeta Fakón,Planeta Fakón,None,Planeta Fakón


#### Discogs

In [47]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID Lookup
####################################################################################################
from lookup import createArtistIDMap
createArtistIDMap(discDC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getSlimArtistDB
discSlimArtistDCDB = getSlimArtistDB(discDC, force=True)
discSlimArtistDCDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Mar 09, 2020 18:22:08 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-discogs-db/metadata/0-Metadata.p  	0 7117
/Volumes/Biggy/Discog/artists-discogs-db/metadata/1-Metadata.p  	1 14494
/Volumes/Biggy/Discog/artists-discogs-db/metadata/10-Metadata.p  	2 21659
/Volumes/Biggy/Discog/artists-discogs-db/metadata/11-Metadata.p  	3 28913
/Volumes/Biggy/Discog/artists-discogs-db/metadata/12-Metadata.p  	4 36183
/Volumes/Biggy/Discog/artists-discogs-db/metadata/13-Metadata.p  	5 43513
/Volumes/Biggy/Discog/artists-discogs-db/metadata/14-Metadata.p  	6 50587
/Volumes/Biggy/Discog/artists-discogs-db/metadata/15-Metadata.p  	7 57790
/Volumes/Biggy/Discog/artists-discogs-db/metadata/16-Metadata.p  	8 64918
/Volumes/Biggy/Discog/artists-discogs-db/metadata/17-Metadata.p  	9 72142
/Volumes/Biggy/Discog/artists-discogs-db/metadata/18-Metadata.p  	10 79361
/Volumes/Biggy/Discog/artists-disc

  --> This file is 26.9MB.
Saved data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToVariations.p
  --> This file is 26.9MB.
Current Time is Mon Mar 09, 2020 18:23:31 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 1.4 minutes.
Current Time is Mon Mar 09, 2020 18:23:31 for 
=================================== Creating Artist DB ===================================
Loading ArtistID Data
Creating Pandas DataFrame for 718315 Artists
	Shape --> (718315, 1)
  Finding Real Artist Name
	Shape --> (718315, 3)
  Removing None Artist
	Shape --> (718315, 3)
  Finding Disc Artist Name
	Shape --> (718315, 4)
DataFrame Shape is (718315, 4)
Current Time is Mon Mar 09, 2020 18:23:52 for Done with 
=================================== Creating Artist DB ===================================
Process [Done with 
=================================== Creating Artist DB ===================================] took 21 seconds.
Saving Master Artist DB File
  --> This file is 57.5MB.


,Name,Artist,Num,DiscArtist
1000,Dave Clarke,Dave Clarke,NaN,Dave Clarke
1000500,Club Pulse,Club Pulse,NaN,Club Pulse
100200,Dike,Dike,NaN,Dike
1002000,Larry Stokes,Larry Stokes,NaN,Larry Stokes
1002600,Gidd Sanchez,Gidd Sanchez,NaN,Gidd Sanchez


## Artist ID --> Albums

#### AllMusic

In [48]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discAM)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistAMAlbumsDB = getArtistAlbumsDB(discAM, force=True)
discArtistAMAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Mar 09, 2020 18:24:00 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/0-MediaMetadata.p 	382       382       1778      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/1-MediaMetadata.p 	746       746       3369      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/10-MediaMetadata.p 	1163      1163      5139      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/11-MediaMetadata.p 	1537      1537      6713      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/12-MediaMetadata.p 	1940      1940      8418      
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/13-MediaMetadata.p 	2322      2322      10071     
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/14-MediaMetadata.p 	2681      2681      11633     
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/15-MediaMetadata.p 	3039      3039      13278     
/Volumes/Biggy/Discog/artists-al

/Volumes/Biggy/Discog/artists-allmusic-db/metadata/80-MediaMetadata.p 	30943     30943     135186    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/81-MediaMetadata.p 	31343     31343     136852    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/82-MediaMetadata.p 	31710     31710     138447    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/83-MediaMetadata.p 	32130     32130     140255    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/84-MediaMetadata.p 	32507     32507     141935    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/85-MediaMetadata.p 	32909     32909     143884    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/86-MediaMetadata.p 	33290     33290     145485    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/87-MediaMetadata.p 	33714     33714     147217    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/88-MediaMetadata.p 	34112     34112     148863    
/Volumes/Biggy/Discog/artists-allmusic-db/metadata/89-MediaMetadata.p 	34510     3

,Albums
0000003100,"{'Albums': {'0000476912': 'Ohla Quem Chega', '..."
0000003500,"{'Albums': {}, 'Single/EP': {'0000934205': 'Ge..."
0000006000,{'Albums': {'0000237785': 'Eyes of the Basilis...
0000007300,"{'Albums': {}, 'Album': {'0000089708': 'Motiva..."
0000015100,"{'Albums': {'0000597370': 'Life Rays', '000035..."


#### MusicBrainz

In [51]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discMB)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistMBAlbumsDB = getArtistAlbumsDB(discMB, force=True)
discArtistMBAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Mar 09, 2020 19:27:54 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/0-MediaMetadata.p 	189       0         759       
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/1-MediaMetadata.p 	403       0         1763      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/10-MediaMetadata.p 	630       0         2792      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/11-MediaMetadata.p 	849       0         3658      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/12-MediaMetadata.p 	1047      0         4356      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/13-MediaMetadata.p 	1253      0         5222      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/14-MediaMetadata.p 	1472      0         5985      
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/15-MediaMetadata.p 	1668      0         6745      
/Volumes

/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/78-MediaMetadata.p 	15986     0         62524     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/79-MediaMetadata.p 	16171     0         63245     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/8-MediaMetadata.p 	16402     0         64300     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/80-MediaMetadata.p 	16633     0         65431     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/81-MediaMetadata.p 	16850     0         66232     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/82-MediaMetadata.p 	17054     0         67013     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/83-MediaMetadata.p 	17262     0         67926     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/84-MediaMetadata.p 	17484     0         68700     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/85-MediaMetadata.p 	17675     0         69529     
/Volumes/Biggy/Discog/artists-musicbrainz-db/metadata/86

,Albums
148150553282083715092252970758494088400,{'Album': {52614366479835895669985691312690064...
153306575689393285671340715814524277900,{'Album': {21489969636009873249112532561634349...
164497588684518360574139427720206551000,{'Album': {25830796384697956307029700203839095...
192627206585753480770469159350953417800,{'Album': {16465593027634309455140682224063378...
196485629247468963190624302831920892400,{'Album': {18602610412619364467964020782245346...


#### Discogs

In [49]:
%load_ext autoreload
%autoreload

####################################################################################################
## Create Artist ID to Album ID Lookup
####################################################################################################
from lookup import createArtistAlbumIDMap
createArtistAlbumIDMap(discDC)


####################################################################################################
## Create Artist ID Pandas DataFrame
####################################################################################################
from masterdb import getArtistAlbumsDB
discArtistDCAlbumsDB = getArtistAlbumsDB(discDC, force=True)
discArtistDCAlbumsDB.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Mon Mar 09, 2020 18:24:47 for Creating Artist DBs
/Volumes/Biggy/Discog/artists-discogs-db/metadata/0-MediaMetadata.p 	7117      3660      19535     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/1-MediaMetadata.p 	14494     7400      40168     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/10-MediaMetadata.p 	21659     11092     60224     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/11-MediaMetadata.p 	28913     14807     80305     
/Volumes/Biggy/Discog/artists-discogs-db/metadata/12-MediaMetadata.p 	36183     18555     100427    
/Volumes/Biggy/Discog/artists-discogs-db/metadata/13-MediaMetadata.p 	43513     22280     120736    
/Volumes/Biggy/Discog/artists-discogs-db/metadata/14-MediaMetadata.p 	50587     25931     140337    
/Volumes/Biggy/Discog/artists-discogs-db/metadata/15-MediaMetadata.p 	57790     29618     160956    
/Volumes/Biggy/Discog/artists-discogs-db

/Volumes/Biggy/Discog/artists-discogs-db/metadata/80-MediaMetadata.p 	574050    293202    1611554   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/81-MediaMetadata.p 	581401    296993    1631906   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/82-MediaMetadata.p 	588680    300741    1652501   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/83-MediaMetadata.p 	595935    304420    1672779   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/84-MediaMetadata.p 	603182    308129    1693017   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/85-MediaMetadata.p 	610338    311777    1713263   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/86-MediaMetadata.p 	617714    315573    1733983   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/87-MediaMetadata.p 	624863    319204    1753946   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/88-MediaMetadata.p 	631965    322780    1774095   
/Volumes/Biggy/Discog/artists-discogs-db/metadata/89-MediaMetadata.p 	639283    326497    1

,Albums
1000,"{'Albums': {'44815': 'Archive One', '44831': '..."
1000500,{'Miscellaneous': {'1178875': 'Peak Controller'}}
100200,"{'Albums': {'50511': 'PottpÃ¼ree'}, 'Singles &..."
1002000,{'Instruments & Performance': {'1002108': 'Are...
1002600,{'Vocals': {'1180905': 'Quiero'}}


***
***
***

## Artist ID --> Genre, Style, Artists Lookup Table

In [22]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discAM)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Feb 21, 2020 13:02:48 for Creating Artist DBs



Saving 0 entries to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToGenre.p

Saving data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToGenre.p
  --> This file is 14B.
Saving 0 entries to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToStyle.p

Saving data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToStyle.p
  --> This file is 14B.
Saving 0 entries to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToCollaborations.p

Saving data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file is 14B.
Saved data to /Volumes/Biggy/Discog/db-allmusic/ArtistIDToCollaborations.p
  --> This file is 14B.
Current Time is Fri Feb 21, 2020 13:02:48 for Done wit

In [23]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createArtistMetadataMap
createArtistMetadataMap(discDC)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current Time is Fri Feb 21, 2020 13:02:48 for Creating Artist DBs
/Volumes/Biggy/Discog/albums-discogs-db/metadata/0-ArtistMetadata.p 	4771
/Volumes/Biggy/Discog/albums-discogs-db/metadata/1-ArtistMetadata.p 	9481
/Volumes/Biggy/Discog/albums-discogs-db/metadata/10-ArtistMetadata.p 	16165
/Volumes/Biggy/Discog/albums-discogs-db/metadata/11-ArtistMetadata.p 	22936
/Volumes/Biggy/Discog/albums-discogs-db/metadata/12-ArtistMetadata.p 	29738
/Volumes/Biggy/Discog/albums-discogs-db/metadata/13-ArtistMetadata.p 	33792
/Volumes/Biggy/Discog/albums-discogs-db/metadata/14-ArtistMetadata.p 	40028
/Volumes/Biggy/Discog/albums-discogs-db/metadata/15-ArtistMetadata.p 	46750
/Volumes/Biggy/Discog/albums-discogs-db/metadata/16-ArtistMetadata.p 	52022
/Volumes/Biggy/Discog/albums-discogs-db/metadata/17-ArtistMetadata.p 	56703
/Volumes/Biggy/Discog/albums-discogs-db/metadata/18-ArtistMetadata.p 	63415
/Volumes/Biggy

  --> This file is 17.4MB.
Saved data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToStyle.p
  --> This file is 17.4MB.
Saving 572424 entries to /Volumes/Biggy/Discog/db-discogs/ArtistIDToCollaborations.p

Saving data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Saved data to /Volumes/Biggy/Discog/db-discogs/ArtistIDToCollaborations.p
  --> This file is 37.7MB.
Current Time is Fri Feb 21, 2020 13:07:38 for Done with Creating Artist DBs
Process [Done with Creating Artist DBs] took 4.8 minutes.


## Album ID --> Name, Ref, Artists Lookup Table

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from lookup import createAlbumIDMap
createAlbumIDMap(disc)

# Master Lookup Tests

In [ ]:
%load_ext autoreload
%autoreload
from lookup import testLookupMaps
testLookupMaps(disc)

# Pandas DB

## Slim Artist DB

In [ ]:
from masterdb import getSlimArtistDB
discSlimArtistDB = getSlimArtistDB(disc)
discSlimArtistDB.head()

## Artist DB

In [ ]:
from masterdb import getArtistDB

In [ ]:
discArtistDB = getArtistDB(disc)
discArtistDB.head()

## Artist Metadata DB

In [ ]:
from masterdb import getArtistMetadataDB

In [ ]:
discArtistMetadataDB = getArtistMetadataDB(disc)
discArtistMetadataDB.head()

## Artist Albums DB

In [ ]:
from masterdb import getArtistAlbumsDB

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc, force=False)
discArtistAlbumsDB.head()

In [ ]:
discArtistAlbumsDB = getArtistAlbumsDB(disc)
discArtistAlbumsDB.head()

## Albums DB

In [ ]:
from masterdb import getAlbumDB

In [ ]:
discAlbumDB = getAlbumDB(disc)
discAlbumDB.head()

## Artist Album ID --> Known Albums

In [ ]:
from masterdb import getArtistAlbumKnownDB

In [ ]:
discArtistAlbumKnownDB = getArtistAlbumKnownDB(discAlbumDB, discArtistAlbumsDB)
discArtistAlbumKnownDB.head()

# Joining Artist ID DataFrame

In [ ]:
from masterdb import createMasterDB

In [ ]:
createMasterDB(disc, discArtistDB, discArtistMetadataDB, discArtistAlbumKnownDB)

In [ ]:
from random import random